In [1]:
import pandas as pd, numpy as np

## Ensemble notebook

### validation score with weight determination (Tune by orders)

In [2]:
def get_unique_session(df, type_mode='orders'):
    test_labels = pd.read_parquet('../data/input/otto/otto-validation/test_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==type_mode]
    df = df.merge(test_labels['session'], how='inner', on='session')
    return df 
        
def get_recall(input_df, type_mode='orders'):
    df = input_df.copy()
    df['session_type'] = df['session'].apply(lambda x: str(x) + f'_{type_mode}')
    df = df.sort_values(['session_type','score'],ascending=[True, False]).reset_index(drop=True)
    

    df['n'] = df.groupby('session_type').cumcount()
    df = df.loc[df.n<20].drop(['n','score','session'],axis=1)
    df['aid'] = df['aid'].astype('int32')
    df = df.groupby('session_type')['aid'].apply(list).reset_index()
    df['labels'] = df['aid'].map(lambda x: ''.join(str(x)[1:-1].split(',')))
    df = df.drop(['aid'],axis=1)

    sub = df.loc[df.session_type.str.contains(type_mode)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    test_labels = pd.read_parquet('../data/input/otto/otto-validation/test_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==type_mode]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['labels'] = test_labels['labels'].fillna('[]')
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    return recall

def set_baseline_to_zero(df):
    # change baseline to zero for ensemble
    session_min_score = df.groupby('session')['score'].min()
    session_min_score = pd.DataFrame({'session': session_min_score.index, 'min_score': session_min_score.values})
    df = df.merge(session_min_score, on='session', how='left')
    df['score'] = df['score'] - df['min_score']
    df = df.drop('min_score', axis=1)
    return df

def get_top_n(df, n):
    df = df.sort_values(['session','score'],ascending=[True, False]).reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<n].drop('n',axis=1)
    return df

In [45]:
# tetsuro order results
topN = 100
#path = '../logs/LB0592/score/valid_score/oof_lgbm_orders.parquet'
path = '../logs/LB0592/score/valid_stack_score/oof_lgbm_orders_stack.parquet' # with stacking
df_tetsuro = pd.read_parquet(path)
df_tetsuro = df_tetsuro[['session', 'aid', 'score']]
df_tetsuro = set_baseline_to_zero(df_tetsuro)
df_tetsuro = get_unique_session(df_tetsuro)
df_tetsuro = get_top_n(df_tetsuro, topN)

In [46]:
df_tetsuro

,session,aid,score
0,11098528,11830,10.336961
1,11098528,1732105,8.312395
2,11098528,588923,7.219396
3,11098528,876129,6.739899
4,11098528,884502,6.372857
...,...,...,...
22785729,12899525,249963,3.187921
22785730,12899525,1103376,3.187447
22785731,12899525,293054,3.182970
22785732,12899525,1449010,3.182535


In [47]:
# gunes
path = './gunes/val_predictions_order.pkl'
df_gunes = np.load(path, allow_pickle=True)
df_gunes = df_gunes.rename(columns={'candidates': 'aid', 'predictions': 'score'})
df_gunes = set_baseline_to_zero(df_gunes)
df_gunes = get_unique_session(df_gunes)
df_gunes = get_top_n(df_gunes, topN)

In [48]:
df_gunes

,session,aid,score
0,11098528,11830,11.204971
1,11098528,1732105,8.226718
2,11098528,876129,7.558273
3,11098528,884502,7.278060
4,11098528,588923,7.166492
...,...,...,...
12456882,12899525,113465,1.618041
12456883,12899525,1683204,1.524922
12456884,12899525,1103376,1.123438
12456885,12899525,38569,0.475823


In [49]:
print('tetsuro original recall: ',get_recall(df_tetsuro) )
print('gunes original recall: ',get_recall(df_gunes) )

tetsuro original recall:  0.6647973367634527
gunes original recall:  0.6591414700784863


In [50]:
# merge
df_merged = df_tetsuro.merge(df_gunes, how = 'outer', on = ['session', 'aid'])
df_merged['score_x']= df_merged['score_x'].fillna(0)
df_merged['score_y']= df_merged['score_y'].fillna(0)

In [51]:
df_merged

,session,aid,score_x,score_y
0,11098528,11830,10.336961,11.204971
1,11098528,1732105,8.312395,8.226718
2,11098528,588923,7.219396,7.166492
3,11098528,876129,6.739899,7.558273
4,11098528,884502,6.372857,7.278060
...,...,...,...,...
18093421,12899525,409671,0.000000,1.875272
18093422,12899525,113465,0.000000,1.618041
18093423,12899525,1683204,0.000000,1.524922
18093424,12899525,38569,0.000000,0.475823


In [52]:
#weight_list = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
weight_list = [0.6, 0.7, 0.8, 0.9]
for w in weight_list:
    print('weight=', w, 1-w)
    df_merged['score'] = df_merged['score_x'] * w  + df_merged['score_y'] * (1-w)
    print('order recall: ',get_recall(df_merged) )
    df_merged = df_merged.drop('score', axis=1)

weight= 0.6 0.4
order recall:  0.6650909822121077
weight= 0.7 0.30000000000000004
order recall:  0.6652792983150496
weight= 0.8 0.19999999999999996
order recall:  0.6654133538459575
weight= 0.9 0.09999999999999998
order recall:  0.6655314503850904


In [ ]:
# memo
# my order v.s. gunes order
# weight= 0 1 order recall:  0.6591414700784863
# weight= 0.2 0.8 order recall:  0.6619279100423552
# weight= 0.4 0.6 order recall:  0.663086532845201
# weight= 0.6 0.4 order recall:  0.6636419057589618 -> Best
# weight= 0.7 0.30000000000000004 order recall:  0.6636387139606068
# weight= 0.8 0.19999999999999996 order recall:  0.6635940287836376
# weight= 1.0 0.0 order recall:  0.6626556400672831

# use top 100 weight= 0.6 0.4 order recall:  0.6636419057589618 -> same, best
# use top70 -> 0.6636036041787025
# use top50 -> 0.6634791240428595
# use top20 -> 0.6630163132813921

# my stacking order v.s. gunes order
# weight= 0.6 0.4 order recall:  0.6650909822121077
# weight= 0.7 0.30000000000000004 order recall:  0.6652792983150496
# weight= 0.8 0.19999999999999996 order recall:  0.6654133538459575
# weight= 0.9 0.09999999999999998 order recall:  0.6655314503850904 -> best


## Submission

In [28]:
# TODO
final_weight_list = [0.7, 0.3]